In [1]:
# Imports
import os
from glob import glob
import pandas as pd
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats.qmc as qmc

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning) # for RuntimeWarning only

In [2]:
def read_INP_data(filename, conc_suffix):
    
    data = pd.read_csv(filename,index_col='temp_bin')
    # If you want to mask any of the data to remove particular runs, you need to add this here.
    # It is easy to filter so that you only get columns that e.g. contain "CAO" or have
    # heights > 2000m if they're in the column name.
    if conc_suffix == '_NT':
        err_suffix = '_err'
    else:
        err_suffix = conc_suffix+'_err'
    raw_concs = data[data.columns[pd.Series(data.columns).str.endswith(conc_suffix)]]
    raw_concs.columns = raw_concs.columns.str.replace(conc_suffix,'')
    logged_concs = np.log(raw_concs)


    raw_errors = data[data.columns[pd.Series(data.columns).str.endswith(err_suffix)]]
    raw_errors.columns = raw_errors.columns.str.replace(err_suffix,'')
    temps = logged_concs.index.values
    return temps, raw_concs, logged_concs, raw_errors

In [3]:
def four_param_fit(T, ln_nu, T_max, a, b):
    # Apply the fit to an array of temperatures T
    return ln_nu + a*(T_max-T)**b

In [4]:
def strip_nans(*args):
    # Strip nans from multiple inputs where NaN values have the same indices in each input.
    nan_mask = np.logical_not(np.isnan(args[0]))
    nanless = []
    for a in args:
        nanless.append(a[nan_mask])
    return nanless

In [5]:
def reduced_chi_squared_test(p, temps, log_concs, weighted_error):
    # Perform reduced chi-squared test for fit parameters p
    modelled = four_param_fit(temps, *p) #*p unlocks the list p so this is like writing temps, p[0], p[1]...
    residual = log_concs - modelled
    chi_sq = np.sum((residual/weighted_error)**2)
    red_chi_sq = chi_sq/(len(log_concs)-len(p))
    return red_chi_sq

In [6]:
# Create 4-dimensional latin hypercube object. Seed chosen for replication of results.
lhc = qmc.LatinHypercube(4, seed=280299)
# Take 100 LH samples
samples = lhc.random(n=100)

In [7]:
def bounded_and_sampled_scipy_4_param(temps, logged_concs, raw_concs, raw_errors, samples, run_id,
                                      nu_range=(-6,-2), plot=False, silent=True):
    """
    Ugly but functional function to find potential best-fits.
    
    Parameters
    ----------
    temps: ndarray
        Temperature bins
    logged_concs: pandas DataFrame
        concentrations with a natural logarithm applied
    raw_concs: pandas DataFrame
        concentrations
    raw_errors: pandas DataFrame
        errors
    samples: qmc Object
        samples chosen in hypercube space
    run_id: string
        identifier of the run
    nu_range: 2-tuple
        (lowest possible nu value, highest possible nu value)
    plot: boolean
        not yet implemented
    silent: boolean
        say when convergence fails (likely to happen often among the 100 samples)

    Returns
    -------
    pandas
        all derived parameters for the run that converged
    """

    # Create dataframe to hold parameters for the run                                      
    run_params = pd.DataFrame(columns=['nu','T_max','a','b','X2','cov'])
    # Strip nans where appropriate as curve_fit doesn't handle them
    nanless_concs, nanless_raw, nanless_errs, nanless_temps = strip_nans(
                                                        logged_concs[run_id].values,
                                                        raw_concs[run_id].values,
                                                        raw_errors[run_id].values,
                                                        temps
    )
    # Weight errors in logarithmic space
    nanless_weights = nanless_errs/nanless_raw

    # Set bounds of the sample space
    lower_bounds = [nu_range[0], max(nanless_temps)-2, 0.3, 0.5]
    upper_bounds = [nu_range[1], max(nanless_temps)+2, 1.05, 1.0]

    # Apply the LH samples to the sample space
    scaled = qmc.scale(samples, lower_bounds, upper_bounds)

    # Loop through all of the samples and attempt to find a best-fit
    for ps in scaled:
        try:
            p, c = opt.curve_fit(four_param_fit, nanless_temps, nanless_concs, p0=ps,
                sigma=nanless_weights, bounds=(lower_bounds,upper_bounds), maxfev=20000)
            # Calculate goodness of fit parameter
            X2 = reduced_chi_squared_test(p, nanless_temps, nanless_concs, nanless_weights)
            # Store in the in the dataframe
            run_params.loc[len(run_params)] = [p[0], p[1], p[2], p[3], X2, c]
        except:
            # Do not fail if convergence can't be found.
            if not silent:
                print('Failed to converge with parameters', ps)
    return run_params

In [8]:
def get_parameter_df(temps, logged_concs, raw_concs, raw_errors, samples, nu_range=(-6,-2)):
    """
    Get a dataframe with the parameters that have the lowest reduced chi-squared for each run.

    Parameters
    ----------
    temps: ndarray
        Temperature bins
    logged_concs: pandas DataFrame
        concentrations with a natural logarithm applied
    raw_concs: pandas DataFrame
        concentrations
    raw_errors: pandas DataFrame
        errors
    samples: qmc Object
        samples chosen in hypercube space
    nu_range: 2-tuple
        (lowest possible nu value, highest possible nu value)

    Returns
    -------
    pandas Dataframe
        best parameters for each run
    """
    bounded_params = {}
    for run in logged_concs:
        # Loop through every run to get all parameters and add to dictionary
        bounded_params[run] = bounded_and_sampled_scipy_4_param(temps, logged_concs, raw_concs,
                                                                raw_errors, samples, run, nu_range)

    # create an empty dataframe to store the lowest X2 values
    lowest_X2_bounded_df = pd.DataFrame(columns=['nu','T_max','a','b','X2','cov'])

    # loop through the dictionary and find the lowest X2 value for each run
    for run_id, df in bounded_params.items():
        df = df.dropna()
        if df.empty:
            # create a new row with NaN values and concatenate it to lowest_X2_df
            lowest_X2_bounded_row = pd.DataFrame([[np.nan]*len(lowest_X2_bounded_df.columns)], 
                                        columns=lowest_X2_bounded_df.columns, 
                                        index=[run_id])
            lowest_X2_bounded_df = pd.concat([lowest_X2_bounded_df, lowest_X2_bounded_row])
        else:
            lowest_X2_bounded_row = df.nsmallest(1, 'X2')
            lowest_X2_bounded_row.index = [run_id]
            lowest_X2_bounded_df = pd.concat([lowest_X2_bounded_df, lowest_X2_bounded_row])

    return lowest_X2_bounded_df

In [9]:
base = '/home/users/erinraif/acao_data/inp_data/'
suffixes = ['_NT', '_nN', '_nS', '_nV']
INP_concs = 'INP_concentrations.csv'
nX_vals = 'nX_calibrated_v3.csv'

In [10]:
files = [INP_concs, nX_vals, nX_vals, nX_vals]
nu_range = [(-6,-2), (-18,-13), (12, 17), (25,33)]

In [11]:
params_df = {}
for s, csv, nus in zip(suffixes, files, nu_range):
    temps, raw_concs, logged_concs, raw_errors = read_INP_data(base+csv, s)
    params = get_parameter_df(temps, logged_concs, raw_concs, raw_errors, samples, nus)
    params_df[s[1:]] = params

In [12]:
params_df['n_INP'] = params_df.pop('NT')


In [13]:
dfs_with_suffixes = []

# Loop through the dictionary and add suffixes to the columns
for key, df in params_df.items():
    df = df.add_suffix(f'_{key}')
    dfs_with_suffixes.append(df)

export_params = pd.concat(dfs_with_suffixes, axis=1)

In [14]:
export_params.index.name = 'unique_ID'
export_params = export_params.loc[:, ~export_params.columns.str.contains('X2|cov')]


In [15]:
export_params.to_csv('/home/users/erinraif/acao_data/metadata/parametrisations_final.csv')

## Median INP and nS parametrisations

In [16]:
def median_4_param(temps, logged_concs, raw_concs, samples, run_id,
                                      nu_range=(-6,-2), plot=False, silent=True):
    """
    Ugly but functional function to find potential best-fits.
    
    Parameters
    ----------
    temps: ndarray
        Temperature bins
    logged_concs: pandas DataFrame
        concentrations with a natural logarithm applied
    raw_concs: pandas DataFrame
        concentrations
    raw_errors: pandas DataFrame
        errors
    samples: qmc Object
        samples chosen in hypercube space
    run_id: string
        identifier of the run
    nu_range: 2-tuple
        (lowest possible nu value, highest possible nu value)
    plot: boolean
        not yet implemented
    silent: boolean
        say when convergence fails (likely to happen often among the 100 samples)

    Returns
    -------
    pandas
        all derived parameters for the run that converged
    """

    # Create dataframe to hold parameters for the run                                      
    run_params = pd.DataFrame(columns=['nu','T_max','a','b','X2','cov'])
    lower_bounds = [nu_range[0], -9, 0.3, 0.5]
    upper_bounds = [nu_range[1], -5, 1.4, 1.0]

    # Apply the LH samples to the sample space
    scaled = qmc.scale(samples, lower_bounds, upper_bounds)

    # Loop through all of the samples and attempt to find a best-fit
    for ps in scaled:
        try:
            p, c = opt.curve_fit(four_param_fit, temps, logged_concs, p0=ps,
                bounds=(lower_bounds,upper_bounds), maxfev=20000)
            # Calculate goodness of fit parameter
            X2 = reduced_chi_squared_test(p, temps, logged_concs, np.ones(len(logged_concs)))
            # Store in the in the dataframe
            run_params.loc[len(run_params)] = [p[0], p[1], p[2], p[3], X2, c]
        except:
            # Do not fail if convergence can't be found.
            if not silent:
                print('Failed to converge with parameters', ps)
    return run_params

In [17]:
temps, raw_concs, logged_concs, raw_errors = read_INP_data(base+INP_concs, '_NT')
med_lc = logged_concs.median(axis=1)
med_rc = raw_concs.median(axis=1)

In [18]:
median_params = median_4_param(temps[(temps < -6) & (temps > -25)], med_lc[(temps < -6) & (temps > -25)], med_rc[(temps < -6) & (temps > -25)], samples, 'median')
min_row_index = median_params['X2'].idxmin()

# Now you can access the entire row using iloc
min_row = median_params.iloc[min_row_index]

In [19]:
min_row

nu                                               -4.212565
T_max                                            -6.955521
a                                                 1.271137
b                                                      0.5
X2                                                0.030058
cov      [[0.831195751090417, -0.21035032112416055, -0....
Name: 9, dtype: object

In [20]:
temps, raw_concs, logged_concs, raw_errors = read_INP_data(base+nX_vals, '_nS')
med_lc = logged_concs.median(axis=1)
med_rc = raw_concs.median(axis=1)
median_params = median_4_param(temps[(temps < -6) & (temps > -24)], med_lc[(temps < -6) & (temps > -24)], med_rc[(temps < -6) & (temps > -24)], samples, 'median', nu_range=(12, 17))
min_row_index = median_params['X2'].idxmin()

# Now you can access the entire row using iloc
min_row = median_params.iloc[min_row_index]

In [21]:
export_params

,nu_nN,T_max_nN,a_nN,b_nN,nu_nS,T_max_nS,a_nS,b_nS,nu_nV,T_max_nV,a_nV,b_nV,nu_n_INP,T_max_n_INP,a_n_INP,b_n_INP
unique_ID,,,,,,,,,,,,,,,,
c273r1t,-14.181367,-6.959897,1.031056,0.500000,15.096196,-6.959748,1.031170,0.500000,30.648234,-6.959251,1.031568,0.500000,-4.521583,-6.959910,1.031049,0.500000
c273r2t,-15.417563,-12.999198,0.615963,0.793139,12.198926,-13.000000,0.505324,0.852425,25.489063,-12.998511,0.617445,0.792410,-4.459991,-12.999211,0.615950,0.793146
c274r1t,-15.171709,-6.643473,1.050000,0.569510,14.800628,-6.643464,1.050000,0.569511,31.697135,-6.643089,1.050000,0.569556,-3.390939,-6.643481,1.050000,0.569510
c274r3t,-17.127144,-4.000000,0.605889,0.818255,12.173406,-6.000000,0.311000,0.955960,25.272001,-6.000000,0.312466,0.963913,-5.485002,-4.000000,0.605883,0.818258
c275r1t,-16.245315,-7.000000,0.981984,0.535260,13.209596,-7.000000,0.964297,0.540007,27.360930,-7.000000,0.995674,0.537636,-3.695110,-7.000000,0.971554,0.538784
c276r2t,-14.681156,-7.000000,1.025641,0.553083,15.001976,-7.000000,1.041656,0.548495,31.292397,-7.000000,1.008424,0.557599,-2.978206,-7.000000,1.029342,0.552928
c276r4t,-16.313304,-10.000000,1.029326,0.570662,13.106303,-10.000000,0.951771,0.591824,27.713443,-9.999999,1.049665,0.564785,-4.162177,-10.000000,1.032194,0.570259
c277r1t,-16.709180,-8.000000,1.049667,0.610791,12.906641,-8.000000,0.980337,0.630145,27.730437,-8.000000,1.049686,0.611418,-4.080409,-8.000000,0.977644,0.631185
c278r2t,-15.885316,-7.000000,0.890209,0.651680,13.608876,-7.000000,0.890151,0.651690,29.210072,-7.000000,0.878181,0.656477,-3.136699,-7.000000,0.885119,0.653918


In [22]:
min_row

nu                                               12.767265
T_max                                            -6.780545
a                                                 1.278689
b                                                      0.5
X2                                                 0.03396
cov      [[2.864196450830608, -1.314195131017727, -1.80...
Name: 5, dtype: object

In [23]:
for col in INP_concs:
    if col.endswith('S'):
        fig, ax = plt.subplots()
        T = np.linspace(-30, -5, 1000)
        
        
    T = np.linspace(

SyntaxError: incomplete input (2767662056.py, line 7)